In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
path = "/Users/mukulagarwal/Desktop/Python_Code/flights_sagemaker_project"
data = "Data"

def get_data(name):
    file_name = f""